In [ ]:
from arcgis.gis import GIS
from arcgis.gis.admin import AGOLAdminManager

# Administering Your GIS Organizations Using ArcGIS API for Python

## Overview

<img src="./img/gettingstarted.jpg" width=700/>

- The ArcGIS ecosystem is vast
- Organizations can have multiple versions of any product or multiple products to manage
- How do you manage this?

## The Way of the Python

<img src="./img/waysofscience.jpg" width=700/>

- The Python API allows administrators to manage, update and control what happens on your server
- Script from your favorite IDE or Notebook environment
- Cross platform support

### What can we do With ArcGIS Online?



<img src="./img/esri-arcgis-online-agol-feature-0.png"/>

## What Can We Manage?

<table  style='font-family:"Courier New", Courier, monospace; font-size:200%' width=50%>
  
  <tr>
    <td>Users</td>
    <td><img src="./img/users.png", width=50/></td>
  </tr>
  <tr>
    <td>Content</td>
    <td><img src="./img/content.png", width=50/></td>
  </tr>
  <tr>
    <td>Credits</td>
    <td><img src="./img/credits.png", width=50/></td>
  </tr>
  <tr>
    <td>Groups</td>
    <td><img src="./img/groups.png" width=50/></td>
  </tr>
 
</table>

## Getting Started

### Understand the `GIS` Object

The `GIS` object is the way users connect to ArcGIS Online and/or Enterprise

- It doesn't matter if you are an administrator of a user, we must start here.

#### Connecting to you `GIS` 

The ArcGIS API for Python support multiple ways of connecting to the `GIS`, which is ArcGIS Online or ArcGIS Enterprise

##### Anonymously

In [ ]:
import pandas as pd
from arcgis.gis import GIS
gis = GIS() #anonymous connection

##### Built-In

- username/password login method
- usersname are case sensitive 

```python
gis = GIS(username='fakeaccount', password='fakepassword')
gis = GIS(url="https://www.mysite.com/portal", username='fakeaccount', password='fakepassword')
```

**Protecting Built-In Credentials**

- using `profiles` will help protect username and passwords.  
- prevents accidental sharing

1. Create a `GIS` object with the extra `profile` parameter

```python
gis = GIS(url="https://www.mysite.com/portal", 
          username='fakeaccount', 
          password='fakepassword', 
          profile='portal_profile')
```

2. Now connect using the `profile`

```python
gis = GIS(profile='portal_profile')
```

**What Happened?**

Instead of keeping your password in plain text, now we leverage the operating system's credential store for the logged in user.  The credentials never get passed on when you use profiles.

## Developer Credentials

- There are two types of credentials:
    - API Keys - provide a long lived token with a set of privileges or actions
    - Application Authentication - a limited set of privileges scopes to a given web application normally

#### Creating API Keys


In [ ]:
from arcgis.gis import GIS
gis = GIS(profile='your_online_profile', trust_env=True)
gis.users.me

##### Access the Administration Endpoint

In [ ]:
from arcgis.gis.admin import AGOLAdminManager
from arcgis.gis.admin._stokenmgr import  TokenPrivilege
import datetime as _dt
admin:AGOLAdminManager = gis.admin
dev_creds = admin.developer_credentials

##### Create a Scoped API Key

In this scenerio, we are going to create an API key to allow a user to login via a token to view and create items.  We will define the privilege scope and set an expiration.

- Provide a `title`, `privileges`, `expiration` and `referer`
- The `expiration` can't be more than 1 year from now

In [ ]:
api_credential = dev_creds.create(title='my first api key',
                                   privileges=[TokenPrivilege.PORTAL_ADMIN_VIEWITEMS, 
                                               TokenPrivilege.PORTAL_USER_CREATEITEM],
                                   referers=['http'],
                                   expiration=_dt.datetime.now() + _dt.timedelta(weeks=20))
api_credential

In [ ]:
token = api_credential.generate_token(slot=1)
token

In [ ]:
GIS(token=token['access_token']).users.me

In [ ]:
api_credential.delete()

##### Other Login Methods

- SAML and OpenID Connect
- Social logins


## User Management

<img src="./img/user_management_tab.png"/>


Users fuel your system. As an administrator your job is to ensure they can put up there content and know the site is reliable and safe.  The Python API is a tool to do just that!

In [ ]:
from arcgis.gis import GIS
gis = GIS(profile='your_online_profile', verify_cert=False)

### Working with Existing Users

In [ ]:
um = gis.users
um

#### Search for Users

In [ ]:
users = um.search("geo*")
users

### Advanced Search for Users

- Advanced Search gives your the full control to look up users.
- When using this search, nothing is there to guide you, so the queries are made by the end user.
- Provides the ability to get user counts and return users as dictionaries

##### How Many Users are in your Organization?

In [ ]:
um.advanced_search(f"accountid:{gis.properties.id}", return_count=True)

##### Write Users to CSV File

In [ ]:
import csv
csv_file = './output.csv'
field_names:list = None
results = um.advanced_search(f"accountid:{gis.properties.id}", as_dict=True,max_users=-1)
field_names = list(results.get("results")[0].keys()) + ['description']
with open(csv_file, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=field_names)
    writer.writeheader()
    for user in results['results']:
        writer.writerow(user)     

#### List User's Groups

In [ ]:
um.user_groups(um.search("dev*")[:2])

#### Access a User's Items

- At 2.4.1+ this returns a Generator
- A generator is a special type of function or expression that creates an iterator, allowing for the generation of values on demand instead of constructing an entire sequence in memory at once. 

In [ ]:
users[0].items()

In [ ]:
for item in users[0].items():
    print(item)
    break

### Creating New Users

In [ ]:
# Check the user types available
um.counts('user_type')

In [ ]:
import uuid
username = f"RUser{uuid.uuid4().hex[:4]}"
password = f"!{uuid.uuid4().hex[:8]}A"
um = gis.users
new_user = um.create(username=username, password=password, 
                     firstname="Dan", lastname="Human", 
                     email='testsadf@esri.com', 
                     role="org_publisher")
new_user

username, password

In [ ]:
new_password = f"!{uuid.uuid4().hex[:8]}A"

#### Reset the Password

In [ ]:
new_user.reset(
    password=password,
    new_password=new_password,
    new_security_question=1,
    new_security_answer=uuid.uuid4().hex[:10],
    reset_by_email=False,
)

In [ ]:
GIS(username=username, 
    password=new_password, 
    verify_cert=False).users.me

#### Deleting the User

In [ ]:
new_user.delete()

### Working with Roles and User Types

<img src="./img/know_your_role.jpg" width=500/>

#### User types

- User type determines the privileges that can be granted to the member through a default or custom role
- Common Roles:
  + viewer, creator and administrator

In [ ]:
from arcgis.gis import GIS
gis = GIS(profile='your_online_profile')
user = gis.users.me

In [ ]:
user.user_types()['id']

In [ ]:
user.update_license_type("GISProfessionalAdvUT")
user.user_types()['id']

In [ ]:
user.update_license_type("creatorUT")

In [ ]:
user.user_types()['id']

#### Working with Roles

- A role defines the set of privileges assigned to a member

**Accessing Role Manager**

In [ ]:
rm = gis.users.roles
rm

**Listing Roles**

In [ ]:
rm.all()

**Check for Existence of a Role**

In [ ]:
rm.exists('DataEditorRole')

In [ ]:
role = rm.create(name="DataEditorRole", 
                 description="Allow to modify service data", 
                 privileges=[
                        "features:user:edit",
                        "features:user:fullEdit",
                        "opendata:user:designateGroup",
                        "portal:admin:viewUsers",
                        "portal:user:createGroup"]
                )
role

**Removing the Role**

In [ ]:
role.delete()

## Managing Content

In [ ]:
from arcgis.gis import GIS
from arcgis.auth.tools._util import detect_proxy
gis = GIS(profile='your_online_profile', verify_cert=False, proxy=detect_proxy(True))

In [ ]:
cm = gis.content
cm

### Working with Content

<img src="./img/content-manager.jpg"/>

**The content manager allows users and administrators to work with, find and manage content**

#### Searching

##### `search` Example

- provides a simple search method
- max items is 10,000
- do not have full control over searches

In [ ]:
cm.search(query="title: battle", item_type="Feature Layer", outside_org=False)

In [ ]:
cm.search(query="title: battle", item_type="Feature Layer", outside_org=True)

##### `advanced_search` Example

- full control searching option
- removed limitations of `search`
- returns items as dictionary, which speeds up searches
- leverage system for simple statistics about content

**How Many Item to Examine?**

In [ ]:
count = cm.advanced_search('title: battle AND  (type:"feature service")', return_count=True)
count

In [ ]:
items = cm.advanced_search('title: battle AND  (type:"feature service")', max_items=count, 
                           sort_field='avgRating', sort_order='desc')['results']
items[10:20]

**Gathering Information from Searches**

- In this demo we will see how much new content was added to the organization in the last 5 days.

In [ ]:
import datetime as _dt

now =_dt.datetime.now(_dt.timezone.utc)
then = now - _dt.timedelta(days=5)


In [ ]:
cm.advanced_search(
    f"created: [{int(then.timestamp()* 1000)} TO {int(now.timestamp()* 1000)}] AND accountid:{gis.properties.id}", 
    return_count=True)

#### Adding and Publishing Content

In [ ]:
import uuid
username = f"UCUser{uuid.uuid4().hex[:3]}"
password = f"!{uuid.uuid4().hex[:6]}A"
user = gis.users.create(username=username,
        password=password,
        firstname=uuid.uuid4().hex[:6],
        lastname=uuid.uuid4().hex[:6],
        email=uuid.uuid4().hex[:6] + "@esri.com",
        role='org_publisher')
user

In [ ]:
list(user.items())

##### Publishing a Table

In this scenerio, the Administrator is going to add and publish a table to the newly created user from the previous steps.

In [ ]:
import io, uuid
import pandas as pd
from arcgis.gis import ItemProperties, ItemTypeEnum
from arcgis.gis import GIS
gis = GIS(profile='your_online_profile')

###### Load the Table into Memory

In [ ]:
buffer = io.StringIO()
df = pd.read_csv("./data/banklist.csv")
df.to_csv(buffer)

###### Get the Destination User's Root Folder

In [ ]:
folder = gis.content.folders.get(owner=user)
folder

###### Populate the Item Properties and Add the Content

In [ ]:
ip = ItemProperties(item_type=ItemTypeEnum.CSV, 
                    title='Failed Banks', 
                    file_name=f"failedbanks{uuid.uuid4().hex[:5]}.csv")
ip

In [ ]:
result = folder.add(item_properties=ip, file=buffer)
result

In [ ]:
item = result.result()
item

###### Analyze the Item and Publish the Table

In [ ]:
analyzed = gis.content.analyze(item=item)
publish_parameters = analyzed['publishParameters']
publish_parameters[
        'name'
    ] = f"Failed_Banks_{uuid.uuid4().hex[:2]}"  # this needs to be updated
publish_parameters['locationType'] = None  # this makes it a hosted table
published_item = item.publish(publish_parameters)

In [ ]:
published_item

In [ ]:
published_item.delete(permanent=True)
item.delete(permanent=True)

##### Add Content to the Current User

In [ ]:
from arcgis.gis import GIS
gis = GIS(profile='your_online_profile')

###### Create a new Folder for the Current User

In [ ]:
folders = gis.content.folders
folder = folders.create("UC2025", exist_ok=True)
folder

###### Create the Item Properties

In [ ]:
shp_data = f"./data/USA_Major_Cities_UC2025.zip"
ip = ItemProperties(item_type=ItemTypeEnum.SHAPEFILE, title='World Cities')
ip

In [ ]:
world_cities_item = folder.add(item_properties=ip, file=shp_data).result()
world_cities_item

In [ ]:
pitem = world_cities_item.publish({'name' : f'world{uuid.uuid4().hex[:4]}cities'})
pitem

##### Reassigning Content 

In [ ]:
for item in user.items():
    print(item)

In [ ]:
folder = gis.content.folders.create("uc_demo_migration_folder", exist_ok=True)
folder

###### Reassign the Feature Service to Created User

In [ ]:
pitem.reassign_to(target_owner=user.username,
                 target_folder= 'uc_demo_migration_folder')

In [ ]:
for item in user.items("uc_demo_migration_folder"):
    print(item)

##### Erase the User

In [ ]:
for item in user.items("uc_demo_migration_folder"):
    item.delete()

In [ ]:
user.delete()

## Viewing All Content

- Administrators have  the ability to view **ALL** content on a given organization from the `gis.admin` endpoint

In [ ]:
# View all Feature Services in a Given Organization
for item in gis.admin.content(item_type=ItemTypeEnum.FEATURE_SERVICE,
                           sort_field='created',
                           order='desc'):
    print(item)
    break

## Metadata 

- Administrators can enable metadata for the organization

In [ ]:
mm = gis.admin.metadata
mm

In [ ]:
mm.enable()

In [ ]:
mm.is_enabled

## Content Migration

### Moving Content Between Organizations

**`clone_items` provides the ability to copy content from one site to another**

- In 2.4.1, the Python API introduced two versions:
    - Offline cloning - this cloning allows for the copying of content in disconnected environments
    - Online cloning - this cloning method requires the direct visualization between the source and destination environment.

##### Offline `cloning` Example

In this scenerio, a user wants to move content from ArcGIS Online to a disconnected Enterprise.

- Please note, applications like dashboards may or may not work due to Enterprise version differences. 

###### Access the Offline Manager Class

In [ ]:
offline_mgr = gis.content.offline

###### Gather the Content for Migration

In [ ]:
items = gis.content.search(f"owner:{gis.users.me.username}")[1:2]
items

###### Export to Local File

In [ ]:
result = offline_mgr.export_items(items=items,
                         output_folder=r"C:\offline_uc_2025_demo")
result

###### Access the Destination GIS and Import

In [ ]:
gis_dest = GIS(profile='your_enterprise_profile', trust_env=True)

In [ ]:
gis_dest.content.offline.import_content(package_path=result)

##### Online `cloning` Example

<img src="./img/clone_items.jpg" />

## Continue onto Reporting and Monitoring 